In [14]:
import os 
FDIR = os.path.dirname(os.path.realpath('__file__'))

from IPython.display import clear_output, display, Image, FileLink, FileLinks, JSON, Markdown, HTML
import ipywidgets as widgets
from shutil import copyfile
from markdown import markdown
from datetime import datetime
import time

#from mf_modules.pydtype_operations import read_json, read_yaml

def _markdown(value='_Markdown_',
              **kwargs):
    """
    a simple template for markdown text input that templates required input
    fields. additional user defined fields can be added as kwargs
    """
    _kwargs = {}
    _kwargs['value'] = markdown(value)  # required field
    _kwargs.update(kwargs)  # user overides
    return widgets.HTML(**_kwargs)
    
class FileController():
    
    def __init__(self, fpth_inputs_options):
        self.out = widgets.Output()
        self.fpth_inputs_options = fpth_inputs_options
        self.fpth_inputs = fpth_inputs_options['working']['fpths'][0]
        self._errors()
        self.form()
        self._init_controls()
    
    @property
    def mf_layout(self):
        return widgets.Layout(
            display='flex',
            flex_flow='row',
            justify_content='flex-start',
            #border='dashed 0.2px green',
            grid_auto_columns='True',
            width='100%',
            align_items='stretch',  
        )
        
    def _errors(self):
        self.errors=[]
        self.iserror = False
        self.errormessage = 'no error'
        
    def form(self):
        
        # button bar
        self.save_changes = widgets.Button(description='save changes',button_style='success',style={'font_weight':'bold'})
        self.revert = widgets.Button(description='revert to last save',button_style='warning',style={'font_weight':'bold'})#,button_style='success'
        self.load_inputs = widgets.ToggleButton(description='inputs from file',button_style='info',style={'font_weight':'bold'})
        self.button_bar = widgets.HBox([self.save_changes, self.revert, self.load_inputs])

        # nested buttons
        template_inputs = self.get_template_inputs()
        project_inputs = self.get_project_inputs()
        options = dict(template_inputs, **project_inputs)
        self.load_button = widgets.Button(description = 'load',icon='fa-upload',style={'font_weight':'bold'})
        self.choose_inputs = widgets.RadioButtons(
            options = options,
            layout = self.mf_layout,
        )
        self.load = widgets.VBox([self.load_button,self.choose_inputs])
        self.inputform = _markdown(value='*TESTING*')
        
        self._init_controls()
    
    def _init_controls(self):
        self.save_changes.on_click(self._save_changes)
        self.load_inputs.observe(self._load_inputs,'value')
        self.load_button.on_click(self._load)
        self.revert.on_click(self._revert)
        
    def _revert(self, sender):
        fpth = self.fpth_inputs
        self.inputform = _markdown(value='revert to inputs in last save of: {0}'.format(fpth))
        self.update_display()
        self.display()

    def _save_changes(self, sender):
        fpth = self.fpth_inputs
        dateTimeObj = datetime.now()
        self.save_timestampStr = dateTimeObj.strftime("%d-%b-%Y %H:%M:%S")
        self.update_display()
        self.display()
        with self.out:
            clear_output()
            display(self.inputform)
            if self.load_inputs.value:
                display(self.load_button)
                display(self.choose_inputs)

    def _load(self,sender):
        self.update_display()
        self.display()
        fpth = self.choose_inputs.value
        try:
            self.iserror = False
            message = 'load input form from: {0}'.format(fpth)
        except:
            self.iserror = True
            self.errormessage = '''__error__ loading : {0}.   
            the file has either been deleted or modified such that it is unreadable'''.format(fpth)

        with self.out:
            clear_output()
            if self.load_inputs.value:
                display(self.load_button)
                display(self.choose_inputs)
                display(Markdown(message))
            if self.iserror:
                display(Markdown(self.errormessage))
            
    def _load_inputs(self,sender):
        self.update_display()
        self.display()
        with self.out:
            clear_output()
            if self.load_inputs.value:
                display(self.load_button)
                display(self.choose_inputs)
                
    def get_project_inputs(self):
        fpths = self.fpth_inputs_options['project']['fpths']
        di = {}
        for fpth in fpths:
            description = 'PROJECT: '+ fpth
            di[description] = fpth
        return di

    def get_template_inputs(self):
        fpths = self.fpth_inputs_options['template']['fpths']
        di = {}
        for fpth in fpths:
            description = 'TEMPLATE: '+ fpth
            di[description] = fpth
        return di

    def update_display(self):
        self.layout = widgets.VBox([self.button_bar, self.inputform])

    def display(self):
        display(widgets.VBox([self.layout, self.out]))
        #display(self.out)

    def _ipython_display_(self):
        self.update_display()
        self.display()
        
        
fpth_inputs_options = {
    'template': {
        'fdir': 'C:\\engDev\\git_mf\\MF_Toolbox\\dev\\mf_scripts\\template_inputs',
        'fpths': ['C:\\engDev\\git_mf\\MF_Toolbox\\dev\\mf_scripts\\template_inputs\\inputs-eplus_pipework_params.csv']
    },
    'project': {
        'fdir': '.\\appdata\\inputs\\archive',
        'fpths': [
            '.\\appdata\\inputs\\inputs-eplus_pipework_params-1.csv',
            '.\\appdata\\inputs\\inputs-eplus_pipework_params.csv',
            '.\\appdata\\inputs\\archive\\20200605-1625-inputs-eplus_pipework_params.csv',
            '.\\appdata\\inputs\\archive\\20200605-1625inputs-eplus_pipework_params.csv',
            '.\\appdata\\inputs\\archive\\20200607_1420-jg-inputs-eplus_pipework_params.csv']
    },
    'working': {
        'fdir': '.\\appdata\\inputs',
        'fpths': ['.\\appdata\\inputs\\inputs-eplus_pipework_params.csv']
    },
}
d = FileController(fpth_inputs_options)
d

In [10]:
d.update_display()